# ISC analysis with levels

> **Remark**. could be the most promising.

`beta_series glm25 subj* nosmooth`

## Main goals

The BOLD signal contains noise. Here, we calculate correlations between subjects to reduce noise and estimate task-relevant signals. Want to find brain regions where the same levels of activity are displayed. Key predictions are that, in the theory encoding region, the ISC should:

1. Be highest for same levels, medium for same games, and lowest for random (shuffled) games
2. Increase over levels of the same game

---

## Outline

0. Load data in
1. Preprocess data
    - Clean up strings
    - Reorder BOLD data based on names
    - 
3. ISC

In [5]:
import h5py
import warnings
import sys 
if not sys.warnoptions:
    warnings.simplefilter("ignore")
import os 
import glob
import time
from copy import deepcopy
import numpy as np
import pandas as pd 

from nilearn import datasets
from nilearn import surface
from nilearn import plotting
from nilearn.input_data import NiftiMasker, NiftiLabelsMasker
import nibabel as nib

from brainiak import image, io
from brainiak.isc import isc, isfc, permutation_isc
from brainiak.isc import compute_summary_statistic
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d 
import seaborn as sns 
import pandas as pd
from importlib import reload 

# import own functions
import utils
from utils import decode_variable, get_in_shape_levels
reload(utils)

%autosave 30
%matplotlib inline
sns.set(style = 'white', context='talk', font_scale=1, rc={"lines.linewidth": 2})

Autosaving every 30 seconds


# 0. Load in data


In [6]:
data_dir = '/Users/Daphne/Desktop/beta_series/' # local directory 
filename = 'beta_series_glm25_subj1_nosmooth.mat' # specify filename

coords_path = '/Users/Daphne/Desktop/beta_series/'
coords_filename = 'coords_nosmooth.mat'

os.path.exists(coords_path) 

True

In [7]:
num_subjects = 8

B_data = []
mask_data = []
Vmask_data = []
names_data = []

for i in range(num_subjects):
    idx = i+1
    
    # change filename to subject #
    data_dir = '/Users/Daphne/Desktop/beta_series/'
    filename = 'beta_series_glm25_subjk_nosmooth.mat'
    filename = filename.replace('k', str(idx))
    
    subject = h5py.File(data_dir+filename,'r') 
    #print(list(subject.keys()))
    print(f'Get data for subject {idx}')
    # load and save data for respective subject
    B = subject['B'].value
    mask = subject['mask'].value
    Vmask = subject['Vmask']
    
    # === decode level names ===
    names = decode_variable(data_dir+filename, 'names')
    
    # append to lists
    B_data.append(B)
    mask_data.append(mask)
    Vmask_data.append(Vmask)
    names_data.append(names)

OSError: Unable to open file (unable to open file: name = '/Users/Daphne/Desktop/beta_series/beta_series_glm25_subj1_nosmooth.mat', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
B_data[0].shape # voxels x levels

In [8]:
mask_data[0].shape

IndexError: list index out of range

In [ ]:
names_data[0].shape # 54 levels (18 blocks x 3 levels per block)

In [ ]:
print(names_data[0][0:10])

In [ ]:
# TODO: get this working
#c = h5py.File(coords_path+coords_filename,'r') 

# 1. Preprocessing

1. Clean the level name strings

2. Reorder the BOLD data

    - split the list of strings `names` into 3 parts: `game, instance, session`.
    - sort the data first by game, then session, then instance (default = quicksort)
    - insert a levels column to the ordered dataframes
    
<font color=red> MEETING: check if the reordering is the same among all matrices. Order: game, session (run), instance.</font>


In [ ]:
ISC_data = []
ordered_dfs = []
clean_names_arr = [] 

for s in range(num_subjects):
    
    print(f'Preprocess fMRI data for subject {s+1}')
    # get the betas and game order from this
    B_s = B_data[s].T # transpose to get [blocks, voxels]
    names_s = names_data[s]
    
    level_names, dfOrdered, B_ordered = get_in_shape_levels(B_s, names_s)
    
    ISC_data.append(B_ordered)
    ordered_dfs.append(dfOrdered)
    clean_names_arr.append(level_names)
    
ISC_arr = np.array(ISC_data)

Check if the ordering went well

In [ ]:
ordered_dfs[0].head(20)

In [ ]:
ordered_dfs[1].head(10)

In [ ]:
ordered_dfs[4].head(10)

In [ ]:
ISC_arr.shape

In [ ]:
ISC_arr = np.swapaxes(ISC_arr, 0, 1) # need to get [TRs, voxels, subjects]
ISC_arr = np.swapaxes(ISC_arr, 1, 2)

ISC_arr.shape # [levels, voxels, subjects]

## 2. ISC

We perform an ISC in two different ways. 

## 2.1 Do ISC and obtain a matrix

We can do an ISC either by treating each row (**block**) as a variable, or by treating each column (**voxel**) as a variable.

### Correlating the voxel patterns of the respective blocks

- In the study from (Chen et al., 2016) they divided the BOLD data obtained from watching and recalling a Sherlock movie up in 50 scenes. This resulted in one vector voxel values for each recalled scene. As such, they compute the correlation between every matching pair of recalled scenes (see img top right).
- In our case, this is equivalent to treating the levels, (i.e. **rows**), as a variable and compute the correlation between each respective level voxel pattern.
- <font color=red> MEETING: interpret results, does this make sense? </font> 

In [ ]:
for i in range(num_subjects):
    
    M_sub = ISC_arr[:,:,i] # take the subject matrix 
    M_rest = np.delete(ISC_arr, i, axis=2) # remove this subjects' data from whole dataset
    
    #print(M_rest.shape)
    # compute average of other matrices
    M_rest_mean = np.mean(M_rest, axis=2)
    
    # now compute correlation between mean matrix & subj matrix
    C_blockpatterns = np.corrcoef(M_sub, M_rest_mean, rowvar=True) # treat rows as variables
    
#     f, ax = plt.subplots(1,1, dpi=100)
#     f.suptitle(f'ISC with subject {i+1}')
#     sns.heatmap(C_blockpatterns, ax=ax, linewidth=0.3, cmap='bwr', vmin=-1, vmax=1);

    mask = np.zeros_like(C_blockpatterns)
    mask[np.triu_indices_from(mask)] = True
    with sns.axes_style("white"):
        f, ax = plt.subplots(1,1, dpi=100)
        f.suptitle(f'ISC with subject {i+1} | shape {C_blockpatterns.shape}')
        ax = sns.heatmap(C_blockpatterns, mask=mask, ax=ax, square=True, cmap='bwr', vmin=-1, vmax=1);  

## 3.2 ISC with Brainiak

- we have `ISC_arr` in the form `[levels, voxels, subjects] = (54, 179595, 8)`

### Visualise the ISC matrix for one subject back on to the brain to see *where* activity is correlated between participants.

In [ ]:
isc_maps = isc(ISC_arr, pairwise=False) # The output of ISC is a voxel by 
                           # participant matrix (showing the result of each individual with the group).

In [ ]:
isc_maps.shape # The output of ISC is subjects x voxels

In [ ]:
isc_maps[0,:]

In [ ]:
data_dir = '/Users/Daphne/Desktop/beta_series/' # local directory

# load in the nift data using nibabel module
brain_nii = nib.load(os.path.join(data_dir, 'mask_nosmooth.nii'))

brain_nii.shape

In [ ]:
# --- choose a subject (0-7) ---
sub = 2

# use the mask to find all the coordinates that represent the brain
coords_sub = np.where(mask_data[sub] == 1) 

# Make the ISC output a volume
isc_vol = np.zeros(brain_nii.shape)
print(isc_vol.shape)

# Map the ISC data for a subject into brain space
isc_vol[coords_sub].shape

isc_maps[sub,:].shape

In [ ]:
# Map the ISC data for a subject into brain space
isc_vol[coords_sub][:-1] = isc_maps[sub,:][:+1] 
print(isc_vol[coords_sub].shape)
print(isc_maps[sub,:].shape)

# make a nii image of the isc map 
isc_nifti = nib.Nifti1Image(isc_vol, brain_nii.affine, brain_nii.header)

print(isc_nifti.shape)

In [ ]:
# Plot the data as a statmap
threshold = .2

f, ax = plt.subplots(1,1, figsize = (12, 5), dpi=100)
plotting.plot_stat_map(
    isc_nifti, 
    threshold=threshold, 
    axes=ax
)
ax.set_title(f'ISC map for subject {sub+1}');